In [16]:
import numpy as np
import tensorflow as tf
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import re
from random import randint
import datetime
from bisect import bisect_left

UNKNOWN_WORD_VECTOR_IDX = 399999
nPFiles = 12500
nNFiles = 12500
ckptInterval = 10000

###############################################
############### HYPERPARAMETERS ###############
###############################################
numDimensions = 300
maxSeqLength = 250 # truncate reviews longer than this
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000 #100K
###############################################

def binarySearchIndex(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

# not the embeddings matrix, but the list
wordsList = np.load('wordsList-lexic-sorted.npy').tolist()
#wordsList = [word.decode('UTF-8') for word in wordsList]
wordVectors = np.load('wordVectors-lexic-sorted.npy')

nWordsInDict = len(wordsList)
print("wordsList (%d words) loaded." % nWordsInDict)
print("wordVectors loaded.")

# dummy call to make sure it's working
hollywoodIndex = wordsList.index('hollywood')
hI2 = binarySearchIndex(wordsList, 'hollywood')
print(hollywoodIndex)
print(hI2)
print(wordsList[hollywoodIndex])

sentence = "hollywood is full of garbage"
wordIdxs = [wordsList.index(word) for word in sentence.split()]

print(wordIdxs)

with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,wordIdxs))
    
positiveFiles = ['positiveReviews/' + f for f in listdir('positiveReviews/') if isfile(join('positiveReviews/', f))]
negativeFiles = ['negativeReviews/' + f for f in listdir('negativeReviews/') if isfile(join('negativeReviews/', f))]

print(len(positiveFiles))
print(len(negativeFiles))


# TODO truncates at 250, this may be ok though
# This will take some time - preprocess and save instead
def generateMatrixOfWordIndices(files, indexMatrix):
    for fileIdx, f in enumerate(files):
        if fileIdx % 1000 == 0:
            print("Processed %d files" % fileIdx)
        with open(f, 'r', encoding='utf-8') as review:
            wordIdx = 0
            # each review only has one line, but w/e
            for line in review.readlines():
                cleanLine = cleanSentences(line)
                split = cleanLine.split()
                for word in split:
                    try:
                        #indexMatrix[fileIdx][wordIdx] = binarySearchIndex(wordsList, word)
                        indexMatrix[fileIdx][wordIdx] = binarySearchIndex(wordsList, word)
                    except ValueError:
                        indexMatrix[wordIdx] = UNKNOWN_WORD_VECTOR_IDX
                    wordIdx = wordIdx + 1
                    if wordIdx == maxSeqLength:
                        break

# TODO dedup with ^^^^
def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize, maxSeqLength],dtype='int32')
    cleanSentence = cleanSentences(sentence)
    split = cleanSentence.split()
    for idxCtr, word in enumerate(split):
        try:
            #sentenceMatrix[0, idxCtr] = binarySearchIndex(wordsList, word)
            sentenceMatrix[0, idxCtr] = binarySearchIndex(wordsList, word)
        except ValueError:
            sentenceMatrix[0, idxCtr] = UNKNOWN_WORD_VECTOR_IDX
    return sentenceMatrix

# Right now, these are ALL TRAINING (see updated bigger dataset)
pMatrix = np.zeros((nPFiles, maxSeqLength), dtype='int32')
nMatrix = np.zeros((nNFiles, maxSeqLength), dtype='int32')

#generateMatrixOfWordIndices(positiveFiles, pMatrix)
#generateMatrixOfWordIndices(negativeFiles, nMatrix)

#np.save('pIDsMatrix.npy', pMatrix)
#np.save('nIDsMatrix.npy', nMatrix)

pMatrix = np.load('pIDsMatrix.npy')
nMatrix = np.load('nIDsMatrix.npy')
print('Loaded pMatrix and nMatrix (index matrices)')

# returns the index matrix and corresponding labels
# so that they can be fed into TensorFlow
def getBatch(pMatrix, nMatrix, isTraining):
    labels = []
    idxMatrix = np.zeros([batchSize, maxSeqLength], dtype='int32')
    for i in range(batchSize):
        randIdx = randint(0,12500-1) # NOTE randint is [a, b] and not [a, b)
        # select a positive example
        if i % 2 == 0:
            idxMatrix[i] = pMatrix[randIdx]
            labels.append([1,0]) # one-hot
        # select a negative example
        else:
            idxMatrix[i] = nMatrix[randIdx]
            labels.append([0,1]) # one-hot
    return idxMatrix, labels


########################################################
################ Begin define RNN model ################
########################################################

ids = np.load('idsMatrix.npy')
print("len IDS: " + str(len(ids)))

tf.reset_default_graph()

# define input and output placeholders
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])
labels = tf.placeholder(tf.float32, [batchSize, numClasses]) # one-hot

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors, input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

# final RNN output weights and biases
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# TensorBoard summary
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)


sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('trained'))
#sess.run(tf.global_variables_initializer())

# TODO start training at 50K iterationa
for i in range(90000, iterations+1):
    batch, batchLabels = getBatch(pMatrix, nMatrix, True)
    sess.run(optimizer, {input_data: batch, labels: batchLabels})


    # update TensorBoard
    if i % 50 == 0:
        summary = sess.run(merged, {input_data: batch, labels: batchLabels})
        writer.add_summary(summary, i)
        
    
    # checkpoint the network
    if i % ckptInterval == 0:
        save_path = saver.save(sess, "trained/pretrained_lstm.ckpt", global_step=i)
        print("Checkpointed at %d iterations." % i)
        
    print("Finished iteration %d." % i)

writer.close()





























"""

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

### The demo itself
#inputText = "John Travolta fails to deliver in this disaster of a film."
inputText = "Christopher Walken delivers in this masterpiece of a film."
inputMatrix = getSentenceMatrix(inputText)

print(inputMatrix)

predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive! (with score %f)" % predictedSentiment[0])
else:
    print("Negative! (with score %f)" % predictedSentiment[1])
"""


print("Done.")

wordsList (400000 words) loaded.
wordVectors loaded.
180682
180682
hollywood
[180682, 192973, 155017, 268046, 157581]
Tensor("embedding_lookup_1:0", shape=(5, 50), dtype=float32)
12500
12500
Loaded pMatrix and nMatrix (index matrices)
len IDS: 25000
INFO:tensorflow:Restoring parameters from trained/pretrained_lstm.ckpt-90000
Checkpointed at 90000 iterations.
Finished iteration 90000.
Finished iteration 90001.
Finished iteration 90002.
Finished iteration 90003.
Finished iteration 90004.
Finished iteration 90005.
Finished iteration 90006.
Finished iteration 90007.
Finished iteration 90008.
Finished iteration 90009.
Finished iteration 90010.
Finished iteration 90011.
Finished iteration 90012.
Finished iteration 90013.
Finished iteration 90014.
Finished iteration 90015.
Finished iteration 90016.
Finished iteration 90017.
Finished iteration 90018.
Finished iteration 90019.
Finished iteration 90020.
Finished iteration 90021.
Finished iteration 90022.
Finished iteration 90023.
Finished iterat

Finished iteration 90302.
Finished iteration 90303.
Finished iteration 90304.
Finished iteration 90305.
Finished iteration 90306.
Finished iteration 90307.
Finished iteration 90308.
Finished iteration 90309.
Finished iteration 90310.
Finished iteration 90311.
Finished iteration 90312.
Finished iteration 90313.
Finished iteration 90314.
Finished iteration 90315.
Finished iteration 90316.
Finished iteration 90317.
Finished iteration 90318.
Finished iteration 90319.
Finished iteration 90320.
Finished iteration 90321.
Finished iteration 90322.
Finished iteration 90323.
Finished iteration 90324.
Finished iteration 90325.
Finished iteration 90326.
Finished iteration 90327.
Finished iteration 90328.
Finished iteration 90329.
Finished iteration 90330.
Finished iteration 90331.
Finished iteration 90332.
Finished iteration 90333.
Finished iteration 90334.
Finished iteration 90335.
Finished iteration 90336.
Finished iteration 90337.
Finished iteration 90338.
Finished iteration 90339.
Finished ite

Finished iteration 90618.
Finished iteration 90619.
Finished iteration 90620.
Finished iteration 90621.
Finished iteration 90622.
Finished iteration 90623.
Finished iteration 90624.
Finished iteration 90625.
Finished iteration 90626.
Finished iteration 90627.
Finished iteration 90628.
Finished iteration 90629.
Finished iteration 90630.
Finished iteration 90631.
Finished iteration 90632.
Finished iteration 90633.
Finished iteration 90634.
Finished iteration 90635.
Finished iteration 90636.
Finished iteration 90637.
Finished iteration 90638.
Finished iteration 90639.
Finished iteration 90640.
Finished iteration 90641.
Finished iteration 90642.
Finished iteration 90643.
Finished iteration 90644.
Finished iteration 90645.
Finished iteration 90646.
Finished iteration 90647.
Finished iteration 90648.
Finished iteration 90649.
Finished iteration 90650.
Finished iteration 90651.
Finished iteration 90652.
Finished iteration 90653.
Finished iteration 90654.
Finished iteration 90655.
Finished ite

Finished iteration 90934.
Finished iteration 90935.
Finished iteration 90936.
Finished iteration 90937.
Finished iteration 90938.
Finished iteration 90939.
Finished iteration 90940.
Finished iteration 90941.
Finished iteration 90942.
Finished iteration 90943.
Finished iteration 90944.
Finished iteration 90945.
Finished iteration 90946.
Finished iteration 90947.
Finished iteration 90948.
Finished iteration 90949.
Finished iteration 90950.
Finished iteration 90951.
Finished iteration 90952.
Finished iteration 90953.
Finished iteration 90954.
Finished iteration 90955.
Finished iteration 90956.
Finished iteration 90957.
Finished iteration 90958.
Finished iteration 90959.
Finished iteration 90960.
Finished iteration 90961.
Finished iteration 90962.
Finished iteration 90963.
Finished iteration 90964.
Finished iteration 90965.
Finished iteration 90966.
Finished iteration 90967.
Finished iteration 90968.
Finished iteration 90969.
Finished iteration 90970.
Finished iteration 90971.
Finished ite

Finished iteration 91250.
Finished iteration 91251.
Finished iteration 91252.
Finished iteration 91253.
Finished iteration 91254.
Finished iteration 91255.
Finished iteration 91256.
Finished iteration 91257.
Finished iteration 91258.
Finished iteration 91259.
Finished iteration 91260.
Finished iteration 91261.
Finished iteration 91262.
Finished iteration 91263.
Finished iteration 91264.
Finished iteration 91265.
Finished iteration 91266.
Finished iteration 91267.
Finished iteration 91268.
Finished iteration 91269.
Finished iteration 91270.
Finished iteration 91271.
Finished iteration 91272.
Finished iteration 91273.
Finished iteration 91274.
Finished iteration 91275.
Finished iteration 91276.
Finished iteration 91277.
Finished iteration 91278.
Finished iteration 91279.
Finished iteration 91280.
Finished iteration 91281.
Finished iteration 91282.
Finished iteration 91283.
Finished iteration 91284.
Finished iteration 91285.
Finished iteration 91286.
Finished iteration 91287.
Finished ite

...

Finished iteration 98520.
Finished iteration 98521.
Finished iteration 98522.
Finished iteration 98523.
Finished iteration 98524.
Finished iteration 98525.
Finished iteration 98526.
Finished iteration 98527.
Finished iteration 98528.
Finished iteration 98529.
Finished iteration 98530.
Finished iteration 98531.
Finished iteration 98532.
Finished iteration 98533.
Finished iteration 98534.
Finished iteration 98535.
Finished iteration 98536.
Finished iteration 98537.
Finished iteration 98538.
Finished iteration 98539.
Finished iteration 98540.
Finished iteration 98541.
Finished iteration 98542.
Finished iteration 98543.
Finished iteration 98544.
Finished iteration 98545.
Finished iteration 98546.
Finished iteration 98547.
Finished iteration 98548.
Finished iteration 98549.
Finished iteration 98550.
Finished iteration 98551.
Finished iteration 98552.
Finished iteration 98553.
Finished iteration 98554.
Finished iteration 98555.
Finished iteration 98556.
Finished iteration 98557.
Finished ite

Finished iteration 98836.
Finished iteration 98837.
Finished iteration 98838.
Finished iteration 98839.
Finished iteration 98840.
Finished iteration 98841.
Finished iteration 98842.
Finished iteration 98843.
Finished iteration 98844.
Finished iteration 98845.
Finished iteration 98846.
Finished iteration 98847.
Finished iteration 98848.
Finished iteration 98849.
Finished iteration 98850.
Finished iteration 98851.
Finished iteration 98852.
Finished iteration 98853.
Finished iteration 98854.
Finished iteration 98855.
Finished iteration 98856.
Finished iteration 98857.
Finished iteration 98858.
Finished iteration 98859.
Finished iteration 98860.
Finished iteration 98861.
Finished iteration 98862.
Finished iteration 98863.
Finished iteration 98864.
Finished iteration 98865.
Finished iteration 98866.
Finished iteration 98867.
Finished iteration 98868.
Finished iteration 98869.
Finished iteration 98870.
Finished iteration 98871.
Finished iteration 98872.
Finished iteration 98873.
Finished ite

Finished iteration 99152.
Finished iteration 99153.
Finished iteration 99154.
Finished iteration 99155.
Finished iteration 99156.
Finished iteration 99157.
Finished iteration 99158.
Finished iteration 99159.
Finished iteration 99160.
Finished iteration 99161.
Finished iteration 99162.
Finished iteration 99163.
Finished iteration 99164.
Finished iteration 99165.
Finished iteration 99166.
Finished iteration 99167.
Finished iteration 99168.
Finished iteration 99169.
Finished iteration 99170.
Finished iteration 99171.
Finished iteration 99172.
Finished iteration 99173.
Finished iteration 99174.
Finished iteration 99175.
Finished iteration 99176.
Finished iteration 99177.
Finished iteration 99178.
Finished iteration 99179.
Finished iteration 99180.
Finished iteration 99181.
Finished iteration 99182.
Finished iteration 99183.
Finished iteration 99184.
Finished iteration 99185.
Finished iteration 99186.
Finished iteration 99187.
Finished iteration 99188.
Finished iteration 99189.
Finished ite

Finished iteration 99468.
Finished iteration 99469.
Finished iteration 99470.
Finished iteration 99471.
Finished iteration 99472.
Finished iteration 99473.
Finished iteration 99474.
Finished iteration 99475.
Finished iteration 99476.
Finished iteration 99477.
Finished iteration 99478.
Finished iteration 99479.
Finished iteration 99480.
Finished iteration 99481.
Finished iteration 99482.
Finished iteration 99483.
Finished iteration 99484.
Finished iteration 99485.
Finished iteration 99486.
Finished iteration 99487.
Finished iteration 99488.
Finished iteration 99489.
Finished iteration 99490.
Finished iteration 99491.
Finished iteration 99492.
Finished iteration 99493.
Finished iteration 99494.
Finished iteration 99495.
Finished iteration 99496.
Finished iteration 99497.
Finished iteration 99498.
Finished iteration 99499.
Finished iteration 99500.
Finished iteration 99501.
Finished iteration 99502.
Finished iteration 99503.
Finished iteration 99504.
Finished iteration 99505.
Finished ite

Finished iteration 99784.
Finished iteration 99785.
Finished iteration 99786.
Finished iteration 99787.
Finished iteration 99788.
Finished iteration 99789.
Finished iteration 99790.
Finished iteration 99791.
Finished iteration 99792.
Finished iteration 99793.
Finished iteration 99794.
Finished iteration 99795.
Finished iteration 99796.
Finished iteration 99797.
Finished iteration 99798.
Finished iteration 99799.
Finished iteration 99800.
Finished iteration 99801.
Finished iteration 99802.
Finished iteration 99803.
Finished iteration 99804.
Finished iteration 99805.
Finished iteration 99806.
Finished iteration 99807.
Finished iteration 99808.
Finished iteration 99809.
Finished iteration 99810.
Finished iteration 99811.
Finished iteration 99812.
Finished iteration 99813.
Finished iteration 99814.
Finished iteration 99815.
Finished iteration 99816.
Finished iteration 99817.
Finished iteration 99818.
Finished iteration 99819.
Finished iteration 99820.
Finished iteration 99821.
Finished ite